## Chap05_회원 탈퇴를 예측

행동 패턴을 분석할 수 있으면 어떤 고객이 탈퇴할지와 같은 예측도 어느 정도 정확하게 할 수 있다.

* 5장에서는 이미 탈퇴한 회원과 계속해서 이용하는 회원의 데이터를 가지고 '의사결정 트리(Decisoin Tree)'라고 부르는 지도학습의 분류 알고리즘을 이용해서 탈퇴를 예측하는 흐름을 정리한다. 
    - 의사결정 트리는 알기 쉽게 원인 분석을 할 수 있다.

* 전제조건 

    - use_log_months.csv : 4장에서 작성한 이용 이력을 연월/고객별로 집계한 데이터
    - customer_join.csv : 3장에서 작성한 이용 이력을 포함한 고객 데이터 

### 테크닉 41 : 데이터를 읽고 이용 데이터를 수정 

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [89]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

머신러닝을 위해 가공 
* 미래를 예측하기 위해 그달과 1개월 전의 이용 이력만으로 데이터 작성 
    - 과거 6개월분의 데이터로부터 이용 횟수를 예측하는 경우, 가입 5개월 이내인 회원의 탈퇴는 예측 불가능 
    - 몇 개월 만에 그만둔 회원도 많기 때문 
    - 이달 과 1개월 전의 이용 횟수를 집계한 데이터 작성

In [90]:
year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame() # 데이터 프레임 객체 생성 

for i in range(1, len(year_months)) :
    tmp = uselog_months.loc[uselog_months["연월"] == year_months[i]]
    tmp.rename(columns = {"count":"count_0"}, inplace = True)
    tmp_before = uselog_months.loc[uselog_months["연월"] == year_months[i-1]]
    del tmp_before["연월"]
    tmp_before.rename(columns = {"count" : "count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on = "customer_id", how = "left")
    uselog = pd.concat([uselog, tmp], ignore_index = True)
uselog.head()
    

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### 테크닉 42 : 탈퇴 전월의 탈퇴 고객 데이터를 작성

왜 end_date 칼럼의 탈퇴 월이 아닌 탈퇴 전월의 데이터를 작성?

* 탈퇴를 예측하는 목적은 탈퇴를 미연에 방지하는 것 
    - 이 스포츠센터에서는 월말까지 탈퇴 신청을 해야 다음 달 말에 탈퇴할 수 있다.
    - 2018년 8월 : 탈퇴 신청, 탈퇴 전월
    - 2018년 9월 : 탈퇴 신청 완료, 탈퇴월
    - 2018년 10월 : 탈퇴 
    

In [91]:
from dateutil.relativedelta import relativedelta # 날짜 연산을 수행할 수 있는 클래스 

exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)) :
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months = 1)
    # "exit_date"가 "end_date"에서 한 달 전의 날짜로 설정
    
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on = ["customer_id", "연월"], how = "left")

print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### 테크닉 43 : 지속 회원의 데이터 작성 

지속 회원은 탈퇴 월이 없기 때문에 어떤 연월의 데이터를 작성해도 된다.

    - 지속 회원 추출 후 ueslog 데이터에 결합 

In [93]:
conti_customer = customer.loc[customer["is_deleted"] == 0]
conti_uselog = pd.merge(uselog, conti_customer, on = ["customer_id"], how = "left")
print(len(conti_customer))
conti_uselog = conti_uselog.dropna(subset = ["name"]) # "name" 컬럼의 결측치 제거, 탈퇴회원 제거 
print(len(conti_uselog))

2842
27422


탈퇴 데이터가 1104개 밖게 없기 때문에 지속 회원 데이터가 27422개를 전부 사용하면 불균형한 데이터가 돼 버림
* 지속 회원 데이터도 회원당 1개가 되게 언더샘플링 진행 
    - 2018년 12월 A씨 중 하나만 선택 
    
* 데이터 섞고 중복 제거

In [94]:
conti_uselog = conti_uselog.sample(frac = 1).reset_index(drop = True)
conti_uselog = conti_uselog.drop_duplicates(subset = "customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201807,PL198544,4,5.0,XXXXXX,C01,M,2016-05-01,NaN,CA1,...,종일,10500.0,일반,4.500000,5.0,5.0,3.0,1.0,2019-04-30,35.0
1,201811,OA854705,3,4.0,XXXXX,C01,M,2016-01-01,NaN,CA1,...,종일,10500.0,일반,4.583333,4.5,7.0,2.0,1.0,2019-04-30,39.0
2,201809,OA911569,10,9.0,XXX,C03,M,2018-08-02,NaN,CA1,...,야간,6000.0,일반,7.875000,7.5,10.0,6.0,1.0,2019-04-30,8.0
3,201807,PL093071,6,2.0,XXXX,C01,F,2016-01-01,NaN,CA1,...,종일,10500.0,일반,4.916667,5.5,8.0,2.0,1.0,2019-04-30,39.0
4,201903,AS625883,6,4.0,XXXX,C02,F,2015-05-01,NaN,CA1,...,주간,7500.0,일반,4.750000,5.0,6.0,2.0,1.0,2019-04-30,47.0


conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True) : "conti_uselog" 데이터프레임을 임의로 섞고(frac=1은 모든 행을 포함하고 순서를 섞는 것을 의미) 새로운 인덱스를 재설정하고(drop=True는 이전 인덱스를 삭제하라는 것을 의미) "conti_uselog"에 다시 할당

* 지속 회원 데이터와 탈퇴 회원 데이터 세로로 결합 

In [95]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index = True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201807,PL198544,4,5.0,XXXXXX,C01,M,2016-05-01,NaT,CA1,...,10500.0,일반,4.500000,5.0,5.0,3.0,1.0,2019-04-30,35.0,NaN
1,201811,OA854705,3,4.0,XXXXX,C01,M,2016-01-01,NaT,CA1,...,10500.0,일반,4.583333,4.5,7.0,2.0,1.0,2019-04-30,39.0,NaN
2,201809,OA911569,10,9.0,XXX,C03,M,2018-08-02,NaT,CA1,...,6000.0,일반,7.875000,7.5,10.0,6.0,1.0,2019-04-30,8.0,NaN
3,201807,PL093071,6,2.0,XXXX,C01,F,2016-01-01,NaT,CA1,...,10500.0,일반,4.916667,5.5,8.0,2.0,1.0,2019-04-30,39.0,NaN
4,201903,AS625883,6,4.0,XXXX,C02,F,2015-05-01,NaT,CA1,...,7500.0,일반,4.750000,5.0,6.0,2.0,1.0,2019-04-30,47.0,NaN


탈퇴 회원 1104개와 지속 고객 2842개가 결합되어 3946개가 됨

    * 유지회원 + 탈퇴회원이 섞인 기본 데이터 작성

### 테크닉 44 : 예측할 달의 재적 기간을 작성 

In [96]:
predict_data["period"] = 0 # "predict_data" 데이터프레임에 "period" 열을 추가하고 모든 행에 0을 할당
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format = "%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)) :
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    # 현재 반복 중인 행의 "now_date"와 "start_date" 사이의 날짜 차이를 계산하고 "delta" 변수에 저장
    predict_data["period"][i] = int(delta.years * 12 + delta.months)
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201807,PL198544,4,5.0,XXXXXX,C01,M,2016-05-01,NaT,CA1,...,4.500000,5.0,5.0,3.0,1.0,2019-04-30,35.0,NaN,26,2018-07-01
1,201811,OA854705,3,4.0,XXXXX,C01,M,2016-01-01,NaT,CA1,...,4.583333,4.5,7.0,2.0,1.0,2019-04-30,39.0,NaN,34,2018-11-01
2,201809,OA911569,10,9.0,XXX,C03,M,2018-08-02,NaT,CA1,...,7.875000,7.5,10.0,6.0,1.0,2019-04-30,8.0,NaN,0,2018-09-01
3,201807,PL093071,6,2.0,XXXX,C01,F,2016-01-01,NaT,CA1,...,4.916667,5.5,8.0,2.0,1.0,2019-04-30,39.0,NaN,30,2018-07-01
4,201903,AS625883,6,4.0,XXXX,C02,F,2015-05-01,NaT,CA1,...,4.750000,5.0,6.0,2.0,1.0,2019-04-30,47.0,NaN,46,2019-03-01


### 테크닉 45 : 결측치 제거 

결측치가 있으면 머신러닝 할 수 없어 제거하거나 채워 넣어야 함

In [97]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               259
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

* count_1, end_date, exit_date 에 결측치 보임 

In [98]:
predict_data = predict_data.dropna(subset = ["count_1"]) # "count_1" 열의 값이 결측치인 (NaN) 행을 제거
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2635
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2635
period                  0
now_date                0
dtype: int64

### 테크닉 46 : 문자열 변수를 처리할 수 있게 가공 

머신러닝을 할 때 가입 캠페인 구분, 회원 구분, 성별과 같은 문자열 데이터는 어떻게 처리해야 할까?

    → 이런 데이터를 카테고리 변수라고 한다. 
    
* 이런 데이터를 활용하기 위해서 앞에서 routine_flg 를 작성한 것처럼 플래그를 생성

    → 이것을 더미변수라고 한다. 

In [99]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,종일,M,5.0,1.0,26,0.0
1,일반,종일,M,4.0,1.0,34,0.0
2,일반,야간,M,9.0,1.0,0,0.0
3,일반,종일,F,2.0,1.0,30,0.0
4,일반,주간,F,4.0,1.0,46,0.0


In [100]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3687 entries, 0 to 3945
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   campaign_name  3687 non-null   object 
 1   class_name     3687 non-null   object 
 2   gender         3687 non-null   object 
 3   count_1        3687 non-null   float64
 4   routine_flg    3687 non-null   float64
 5   period         3687 non-null   int64  
 6   is_deleted     3687 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 230.4+ KB


* 설명변수 
    - 1개월 전의 이용 횟수 : count_1
    - 카테고리 변수 : campaign_name, class_name, gender
    - 정기 이용 여부 플래그 : routine_flg
    
* 지도학습의 분류로 진행 
    - 분류는 회귀와는 달리, 탈퇴/유지와 같은 이산치를 목적 변수로 사용 

In [101]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,5.0,1.0,26,0.0,True,False,False,False,True,False,False,True
1,4.0,1.0,34,0.0,True,False,False,False,True,False,False,True
2,9.0,1.0,0,0.0,True,False,False,True,False,False,False,True
3,2.0,1.0,30,0.0,True,False,False,False,True,False,True,False
4,4.0,1.0,46,0.0,True,False,False,False,False,True,True,False


* 남성, 여성을 더미 변수로 만들 때 여성 컬럼(gender_F)이 1이면 여성을 의미 0이면 남성을 의미 
* 회원구분도 class_name_종일과 class_name_주간이 0이면 class_name_야간이 1인 것과 같다

    - 각 더미 변수에서 하나씩 지워야 함
    - campaign_name_일반, class_name_야간, gender_M 삭제

In [102]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,5.0,1.0,26,0.0,False,False,True,False,False
1,4.0,1.0,34,0.0,False,False,True,False,False
2,9.0,1.0,0,0.0,False,False,False,False,False
3,2.0,1.0,30,0.0,False,False,True,False,True
4,4.0,1.0,46,0.0,False,False,False,True,True


### 테크닉 47 : 의사결정 트리를 사용해서 탈퇴 예측 모델 구축

의사결정 트리는 지도학습의 분류 알고리즘

In [105]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

# 탈퇴 데이터와 유지데이터를 동일한 비율로 추출
exit = predict_data.loc[predict_data["is_deleted"] == 1]
conti = predict_data.loc[predict_data["is_deleted"] == 0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index = True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state = 0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred) # 1은 탈퇴, 0은 유지

[1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0.
 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0.

In [106]:
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
679,1.0,1.0
1332,0.0,0.0
1822,0.0,0.0
1470,0.0,0.0
314,1.0,1.0


### 테크닉 48 : 예측 모델을 평가하고 모델을 튜닝

results_test 데이터 집계 후 정답률 계산 

     - 정답 수는 results_test 데이터의 y_test와 y_pred가 일치하는 개수
     - 이 개수를 전체 데이터 개수로 나누면 정답률 계산 

In [107]:
correct = len(results_test.loc[results_test["y_test"] == results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8821292775665399


정답률 : 88%

각각의 데이터를 이용했을 때의 정확도 계산 

In [108]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8821292775665399
0.9797211660329531


* 평가용 데이터 예측 성능 : 88%
* 학습용 데이터 예측 성능 : 97%

    - 학습용 데이터에 너무 맞춘 과적합 경향 \
    
* 이런 경우에는 데이터 늘리기, 변수 재검토, 모델의 파라미터 변경과 같은 방법을 적용해 가면서 이상적인 모델로 만들어감 

In [111]:
X = pd.concat([exit, conti], ignore_index = True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state = 0, max_depth = 5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9296577946768061
0.9239543726235742


* 평가용, 학습용 데이터 모두 92% 

* max_depth를 지정하면 모델이 단순해져 미지의 데이터에도 대응할 수 있는 좋은 모델 생성 가능

### 테크닉 49 : 모델에 기여하는 변수 확인

변수와 중요도를 데이터프레임에 저장
    
    - 변수의 기여율을 출력하는 함수가 조금 다름
    

In [112]:
importance = pd.DataFrame({"feature_names" : X.columns, "coefficient" :model.feature_importances_ })
importance

,feature_names,coefficient
0,count_1,0.337713
1,routine_flg,0.136554
2,period,0.521628
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.004106
5,class_name_종일,0.000000
6,class_name_주간,0.000000
7,gender_F,0.000000


* 회귀와는 달리 model.feature_importances_로 변수 중요도 확인 가능 

    - 실행 결과로 보면 1개월 전의 이용 횟수, 정기 이용 여부, 재적 기간이 기여하고 있음 확인 

### 테크닉 50 : 회원 탈퇴 예측 

모델을 작성할 때 설명 변수를 참고로 예측을 위한 데이터 작성 

* 먼저 1개월 전의 이용 횟수를 3회
* 정기 이용
* 재적 기간 10
* 캠페인 구분은 가입비 무료
* 회원 구분은 종일
* 성별은 남성

으로 해서 작성 

In [113]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

변수가 정의되면 데이터 가공 

In [115]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
    
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
    
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
    
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [116]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]
